## MNST image download

In [9]:
import torch.utils.data as data_utils
from torchvision import datasets, transforms
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests
from tqdm import tqdm

In [2]:
dataset = datasets.MNIST('../datasets',
                      train=True,
                      download=True,
                      transform=None)

In [3]:
len(dataset), dataset[0][0]

(60000, <PIL.Image.Image image mode=L size=28x28>)

In [4]:
imgs = [dataset[0][0], dataset[1][0]]
imgs[0]

In [5]:
imgs[1]

In [6]:
len(dataset)

60000

## Investigate DINO V2 from Hugging face

In [14]:
# url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
# image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-small')
model = AutoModel.from_pretrained('facebook/dinov2-small')
model.cuda()
inputs = processor(images=imgs, return_tensors="pt").to("cuda:0", non_blocking=True)
outputs = model(**inputs)
last_hidden_states = outputs[0]

In [15]:
outputs.last_hidden_state.shape, last_hidden_states.shape

(torch.Size([2, 257, 384]), torch.Size([2, 257, 384]))

In [10]:
inputs = processor(images=imgs, return_tensors="pt")

In [11]:
inputs["pixel_values"].shape

torch.Size([2, 3, 224, 224])

### Batch class encoding using DINO v2

In [20]:
dataset = datasets.MNIST('../datasets',
                      train=True,
                      download=True,
                      transform=None)
batch_size = 128
n = len(dataset)
n_batch = (n + batch_size - 1) // batch_size
embedding_list = []
for batch_i in tqdm(range(n_batch)):
    imgs = [dataset[i_][0] for i_ in range(batch_i * batch_size, (batch_i+1) * batch_size) if i_ < n]
    inputs = processor(images=imgs, return_tensors="pt").to("cuda:0", non_blocking=True)
    with torch.no_grad():
        last_hidden_states = model(**inputs)[0]
        embedding_list.append(last_hidden_states[:, 0, :].cpu()) # extract cls token embedding 
torch.save(torch.concat(embedding_list), "../datasets/mnst_train_dinov2_small.pt")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 469/469 [06:49<00:00,  1.15it/s]


In [24]:
dataset = datasets.MNIST('../datasets',
                      train=False,
                      download=True,
                      transform=None)
batch_size = 128
n = len(dataset)
n_batch = (n + batch_size - 1) // batch_size
embedding_list = []
for batch_i in tqdm(range(n_batch)):
    imgs = [dataset[i_][0] for i_ in range(batch_i * batch_size, (batch_i+1) * batch_size) if i_ < n]
    inputs = processor(images=imgs, return_tensors="pt").to("cuda:0", non_blocking=True)
    with torch.no_grad():
        last_hidden_states = model(**inputs)[0]
        embedding_list.append(last_hidden_states[:, 0, :].cpu()) # extract cls token embedding 
torch.save(torch.concat(embedding_list), "../datasets/mnst_test_dinov2_small.pt")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 79/79 [01:06<00:00,  1.19it/s]


In [14]:
dataset = datasets.MNIST('../datasets',
                      train=True,
                      download=True,
                      transform=None)
labels = [dataset[i_][1] for i_ in range(len(dataset))]
torch.save(torch.tensor(labels), "../datasets/mnst_train_labels.pt")

In [15]:
dataset = datasets.MNIST('../datasets',
                      train=False,
                      download=True,
                      transform=None)
labels = [dataset[i_][1] for i_ in range(len(dataset))]
torch.save(torch.tensor(labels), "../datasets/mnst_test_labels.pt")

In [68]:
train_embedding = torch.load("../datasets/mnst_train_dinov2_small.pt")
train_labels = torch.load("../datasets/mnst_train_labels.pt")